In [1]:
import numpy as np
import random

In [69]:
class Board():
    def __init__(self, size=3):
        self.size = size
        self._state = np.zeros((size, size), dtype=int)
        #self.marker_to_value = {'-': 0, 'X': 1, 'O': 2}
        self.value_to_marker = {0: '-', 1: 'X', 2: 'O'}
        
    @property
    def state(self):
        """Get the current state of the board."""
        return self._state
    
    def moves_available(self):
        return list(zip(*np.where(self.state==0)))
    
    @staticmethod
    def play_move(board, move, marker):
        marker_to_value = {'-': 0, 'X': 1, 'O': 2}
        # fail if move is not available
        assert board.state[move[0]][move[1]] == 0
        assert marker in ['X', 'O']
        board.state[move[0]][move[1]] = marker_to_value[marker]
        return board.state
    
    def draw(self):
        """
        Prints the board
        """
        for m in range(self.size):
            print('|'.join([self.value_to_marker[self.state[m][n]] for n in range(self.size)]))
            if m != self.size-1:
                print('-'*((self.size*2)-1))
        
    def is_finished(self) -> bool:
        """
        Check of board is in a winning state
        Return 0 if not a winning state, otherwise
        Return the integer of the winning player (1 or 2)
        """
        
        rows = [self.state[i,:] for i in range(self.size)]
        cols = [self.state[:,j] for j in range(self.size)]
        diag = [np.array([self.state[i,i] for i in range(self.size)])]
        cross_diag = [np.array([self.state[(self.size-1)-i,i] for i in range(self.size)])]
        lanes = np.concatenate((rows, cols, diag, cross_diag))
        for lane in lanes:
            if set(lane) == {1}:
                print("player 1 wins: ", lane)
                return True
            if set(lane) == {2}:
                print("player 2 wins: ", lane)
                return True
        
        # check for draw
        if np.all(board.state!=0):
            print('Draw!')
            return False
        
        # game still in progress
        return False

In [70]:
class RandomPlayer():

    def __init__(self, marker):
        self.marker = marker
        
    def move(self, board):
        
        moves_available = board.moves_available()
        move = random.choice(moves_available)
        return move

In [76]:
board = Board()

player1 = RandomPlayer('X')
player2 = RandomPlayer('O')

turns = 0
active_player = player1
inactive_player = player2
while not board.is_finished():
    move = active_player.move(board)
    board.play_move(board=board, move=move, marker=active_player.marker)
    #board.draw()
    turns += 1
    if turns > 20:
        break
    # switch players
    active_player, inactive_player = inactive_player, active_player
    
board.draw()

player 1 wins:  [1 1 1]
X|X|O
-----
X|O|O
-----
X|O|X


In [29]:
###############################

In [ ]:
def search(board, max_depth=3) -> DiGraph:
    """
    Run game simulations from current game state to a maximum number
    of moves ahead (max_depth)
    Return the graph of possible moves and outcomes
    state = [(1,1), (1,1)]
    Assume first player is play to be maximized
    """

    n = 0 # node label which also serves as a node counter
    depth = 0
    
    G = nx.DiGraph()
    G.add_node(0, winner=False, player='max', state=board.state, board=board.draw())
    
    # First branch in look ahead
    newleavelist=[]
    parent_node = n
    parent_board = board

    for move in parent_board.moves_available():
        
        # Do move
        new_board = parent_board.update_board(Move(player=0, move=move))
        
        # Add move node to graph
        n=n+1
        G.add_node(n, winner=new_board.is_winner, player=1, board=new_board.state, board_p = new_board.display)
        G.add_edge(parent_node, n, move=move)
        if new_board.is_winner:
            continue
        newleavelist.append(n)
    
    depth=1
    # subsequent branches
    while depth < max_depth:
        leavelist = newleavelist[:]
        newleavelist = []
        for leave in leavelist: 
            # Get parent board
            parent_board = Board(G.nodes[leave]['board'][0], G.nodes[leave]['board'][1])
            for move in ALL_MOVES:
                moves_available = parent_board.moves_available(player=depth%2)
                if move not in moves_available:
                    continue
                # Do move
                new_board = parent_board.update_board(Move(player=depth%2, move=move))
                # Add move node to graph
                n=n+1
                G.add_node(n, winner=new_board.is_winner, player=1-depth%2, 
                           board=new_board.state, board_p=new_board.display)
                G.add_edge(leave, n, move=move)
                if new_board.is_winner:
                    continue
                    
                newleavelist.append(n)
        depth=depth+1
    return G    


In [ ]:
# class Game():
#     def __init__(self, player1, player2):
#         self.player1 = player1
#         self.player2 = player2
#         self._active_player = player1
#         self._inactive_player = player2
#         self._turns = 1
#         self.player1.idx = 1 # test that these can be switched without affecting game score
#         self.player2.idx = 0
        
#     @property
#     def active_player(self):
#         return self._active_player
    
#     @property
#     def inactive_player(self):
#         return self._inactive_player
    
#     @property
#     def turns(self):
#         return self._turns
    
#     def end_of_turn(self):
#         '''Switch players and increment turn count'''
#         self._active_player, self._inactive_player = self._inactive_player, self._active_player
#         self._turns += 1